In [1]:
!pip install qiskit
!pip install qiskit-aer
!pip install numpy
!pip install matplotlib
!pip install scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 56.4 MB/s eta 0:00:00


In [31]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit.quantum_info import state_fidelity, Operator, DensityMatrix
from qiskit_aer.noise import thermal_relaxation_error, depolarizing_error
import numpy as np
from typing import List, Tuple, Optional
from qiskit.primitives import BackendSamplerV2
from qiskit.quantum_info import partial_trace

In [32]:
class QuantumCommunicationChannel:
    def __init__(self, distance , num_repeaters):
        self.distance = distance
        self.num_repeaters = num_repeaters
        self.segment_length = distance / (num_repeaters + 1) if num_repeaters > 0 else distance

        self.attenuation_coefficient = 0.2  #0.2 dB/km
        self.coherence_time = 1e-4
        self.shots = 1024

        self.simulator = AerSimulator()
        self.noise_model = self.create_noise_model()

    def create_noise_model(self):
        noise_model = NoiseModel()

        error_prob = 1 - np.exp(-self.segment_length * self.attenuation_coefficient / 10)
        print(f"Error probability: {error_prob}")

        error = depolarizing_error(error_prob, 1)
        noise_model.add_all_qubit_quantum_error(error, ['id', 'x', 'h'])

        two_qubit_error_prob = min(2 * error_prob, 1.0)  #Maximo error de 100%
        dep_error_2q = depolarizing_error(two_qubit_error_prob, 2)
        noise_model.add_all_qubit_quantum_error(dep_error_2q, ['cx'])

        t1 = self.coherence_time
        t2 = self.coherence_time / 2
        error = thermal_relaxation_error(t1, t2, 0)
        noise_model.add_all_qubit_quantum_error(error, ['id', 'x', 'h'])

        return noise_model

    def create_bell_pair(self):
        qr = QuantumRegister(2, 'q')
        cr = ClassicalRegister(2, 'c')
        circuit = QuantumCircuit(qr, cr)

        circuit.h(qr[0])
        circuit.cx(qr[0], qr[1])

        return circuit

    def perform_entanglement_swapping(self, qc, q1, q2, q3, q4, cr_offset):
        qc.cx(q2, q3)
        qc.h(q2)

        qc.measure(q2, cr_offset)
        qc.measure(q3, cr_offset + 1)

        with qc.if_test((cr_offset, True)):
            qc.x(q1)
            qc.x(q4)
        with qc.if_test((cr_offset + 1, True)):
            qc.z(q1)
            qc.z(q4)

    def calculate_bell_fidelity(self, counts):
        total_shots = sum(counts.values())
        print("")
        correct_z = 0

        for outcome, count in counts.items():
            endpoint_measurements = outcome[-2:]

            if endpoint_measurements in ['00', '11']:
                correct_z += 1

        fidelity = correct_z / total_shots

        return fidelity

    def simulate_communication(self):
        num_qubits = 2 * (self.num_repeaters + 1)
        qr = QuantumRegister(num_qubits, 'q')
        cr = ClassicalRegister(num_qubits, 'c')
        qc = QuantumCircuit(qr, cr)

        for i in range(0, num_qubits - 1, 2):
            bell_circuit = self.create_bell_pair()
            qc.compose(bell_circuit, [i, i+1], inplace=True)

        if self.num_repeaters > 0:
            for i in range(1, self.num_repeaters + 1):
                self.perform_entanglement_swapping(qc, (i-1)*2, (i-1)*2+1, i*2, i*2+1, (i-1)*2)


        qc.measure(0, cr.size - 2)
        qc.measure(num_qubits - 1, cr.size - 1)

        self.simulator.set_options(noise_model=self.noise_model)

        result = self.simulator.run(qc, shots=self.shots).result()
        counts = result.get_counts()

        fidelity = self.calculate_bell_fidelity(counts)

        return fidelity, counts

In [33]:
def analyze_performance(distances, repeater_configs):
    results = []

    for distance in distances:
        for num_repeaters in repeater_configs:
            print(f"Distance: {distance}km, Repeaters: {num_repeaters}")
            channel = QuantumCommunicationChannel(distance, num_repeaters)
            #print("Channel created!")
            fidelity, counts = channel.simulate_communication()
            #print("Simulation complete!")
            result = {
                'distance': distance,
                'num_repeaters': num_repeaters,
                'fidelity': fidelity,
                'counts': counts
            }
            results.append(result)

            print(f"Fidelity: {fidelity:.4f}")
            #print(f"Measurement counts: {counts}")

        print("\n" + "="*50)

    return results

In [34]:
if __name__ == "__main__":
    test_distances = [1, 5, 10, 50]
    test_repeater_configs = [0, 1, 2, 4, 5]

    results = analyze_performance(test_distances, test_repeater_configs)

Distance: 1km, Repeaters: 0
Error probability: 0.019801326693244747

Fidelity: 0.0020
Distance: 1km, Repeaters: 1
Error probability: 0.009950166250831893

Fidelity: 0.0078
Distance: 1km, Repeaters: 2
Error probability: 0.006644493744965563

Fidelity: 0.0264
Distance: 1km, Repeaters: 4
Error probability: 0.003992010656008516

Fidelity: 0.2432
Distance: 1km, Repeaters: 5
Error probability: 0.0033277839454767255

Fidelity: 0.4004

Distance: 5km, Repeaters: 0
Error probability: 0.09516258196404048

Fidelity: 0.0020
Distance: 5km, Repeaters: 1
Error probability: 0.048770575499285984

Fidelity: 0.0078
Distance: 5km, Repeaters: 2
Error probability: 0.0327838995179941

Fidelity: 0.0303
Distance: 5km, Repeaters: 4
Error probability: 0.019801326693244747

Fidelity: 0.2852
Distance: 5km, Repeaters: 5
Error probability: 0.01652854617838251

Fidelity: 0.4199

Distance: 10km, Repeaters: 0
Error probability: 0.18126924692201818

Fidelity: 0.0020
Distance: 10km, Repeaters: 1
Error probability: 0.09516